In [2]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.3 MB/s eta 0:00:00


In [5]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from datetime import datetime, date, timedelta
import datetime as dt
import pandas as pd
from datetime import datetime, date, timedelta
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import re

In [6]:
# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords and stemmer for English
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Preprocessing function for each comment
def normalize_document(doc):
    # Remove non-alphabetic characters (keep only letters)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, flags=re.I | re.A)
    doc = doc.lower().strip()

    # Tokenize the document
    tokens = word_tokenize(doc)

    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]

    # Apply stemming
    tokens = [stemmer.stem(token) for token in tokens]

    # Rebuild the document from the processed tokens
    doc = ' '.join(tokens)
    return doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Applying NLP on comments column

In [12]:
nltk.download('punkt_tab')
full_df =  pd.read_csv("full_dataset_updated.csv",on_bad_lines='skip')
full_df = full_df.dropna(subset=['comment_body'])
full_df['post_created'] = pd.to_datetime(full_df['post_created'], format='%Y-%m-%d %H:%M:%S')
full_df = full_df[full_df["comment_body"]!="[removed]"]
full_df = full_df[full_df["comment_body"]!="[deleted]"]
full_df['processed_comments'] = full_df['comment_body'].apply(normalize_document)
full_df[['comment_body', 'processed_comments']].head()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,comment_body,processed_comments
0,It’s the country i live in and it’s natural fo...,countri live natur compar leader want compar u...
1,https://en.wikipedia.org/wiki/North_American_C...,httpsenwikipediaorgwikinorthamericanchargingst...
2,Very good point.,good point
3,If they can stay alive long enough. Most of th...,stay aliv long enough brand burn ton cash pole...
4,Google software 🤣\nIt won't fix the laggy sinc...,googl softwar wont fix laggi sinc gooogl cant ...


In [13]:
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]
company_dfs = {company: full_df[full_df['company'] == company] for company in full_df['company'].unique()}
for company_name in keywords:
    company_df = company_dfs.get(company_name)

    globals()[company_name] = company_df


In [14]:
globals()['microsoft'].head()

,post_title,post_score,post_id,post_num_comments,post_created,post_body,comment_id,comment_body,comment_score,company,processed_comments
168,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt1g4fi,Clearly... \n\nWaymo actually has a fully auto...,4,microsoft,clear waymo actual fulli autonom ride share se...
169,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kszx1da,ChatGPT 4 can search the web with the paid ver...,18,microsoft,chatgpt search web paid version
170,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt06obu,>Just remember all of ChatGPT’s data is over 2...,4,microsoft,rememb chatgpt data year old actual true reall...
171,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kszy5m0,"No it isn’t, GPT4’s latest training data is f...",17,microsoft,isnt gpts latest train data april brows web li...
172,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt06d1j,">I've used Chat GPT sometimes, but damn it's b...",3,microsoft,ive use chat gpt sometim damn bad free paid ve...


### Using VADER (Better for Social Media)

In [15]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    scores = sia.polarity_scores(text)
    return scores['compound']


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [16]:
for company in keywords:
    if company in globals():
        df = globals()[company_name].copy()
        if df is None:
            print(f"{company_name} DataFrame is None.")
            continue
        df['sentiment_score'] = df['processed_comments'].apply(analyze_sentiment_vader)

        # Classify sentiment
        df['sentiment'] = df['sentiment_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

        #df[['processed_comments', 'sentiment_score', 'sentiment']].head()
        globals()[company_name] = df

In [ ]:
globals()['meta'][['processed_comments', 'sentiment_score', 'sentiment']].head()


,processed_comments,sentiment_score,sentiment
728,didnt say didnt tech compani definit counterin...,0.5574,positive
729,eu compani slight less vicious manag us busi c...,-0.6378,negative
730,thank ask plutu sound vagu,0.3612,positive
731,read post,0.0000,neutral
732,want offer subscript app store make easier use...,0.6124,positive


###  Using Hugging Face Transformers (Advanced Model)

In [17]:
!pip install transformers torch

In [18]:
!pip install emoji==0.6.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=a6a98ee3cd2eaec7c28097dc22487394eb081352660dd8d629479dac5435c46e
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [20]:
import torch

if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available: Tesla T4


In [22]:
from transformers import pipeline
sentiment_analyzer = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis",device=0)

for company in keywords:
    if company in globals():
        df = globals()[company]
        if df is None:
            print(f"{company_name} DataFrame is None.")
            continue
        df['sentiment_results'] = df['processed_comments'].apply(lambda x: sentiment_analyzer(x[:128]))

        df['sentiment'] = df['sentiment_results'].apply(lambda x: x[0]['label'])
        df['confidence'] = df['sentiment_results'].apply(lambda x: x[0]['score'])

        globals()[company] = df



MARA Holdings DataFrame is None.


<ipython-input-22-f63d75e1cd5e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_results'] = df['processed_comments'].apply(lambda x: sentiment_analyzer(x[:128]))
<ipython-input-22-f63d75e1cd5e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment_results'].apply(lambda x: x[0]['label'])
<ipython-input-22-f63d75e1cd5e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [25]:
globals()['Uber'][['processed_comments', 'sentiment', 'confidence']].head()

,processed_comments,sentiment,confidence
1125,stock great compani make ton moneyub freight u...,NEG,0.925684
1126,uber go buy instacart month,NEU,0.920818
1127,advantag destroy taxi busi freight,NEG,0.923880
1128,absolut worst would trailer dock day load alwa...,NEG,0.983154
9227,jump geofenc geofenc distanc driver front seat...,NEU,0.968688


In [32]:
for company in keywords:
    if company in globals():
        df = globals()[company]
        if df is None:
            print(f"{company} DataFrame is None.")
            continue
        print(f"{company}: {df.shape[0]} comments")
        print(df['sentiment'].value_counts())

Uber: 1317 comments
sentiment
NEU    941
NEG    249
POS    127
Name: count, dtype: int64
youtube: 2083 comments
sentiment
NEU    1415
NEG     442
POS     226
Name: count, dtype: int64
meta: 6469 comments
sentiment
NEU    4416
NEG    1461
POS     592
Name: count, dtype: int64
apple: 7188 comments
sentiment
NEU    4878
NEG    1641
POS     669
Name: count, dtype: int64
nvidia: 8637 comments
sentiment
NEU    6282
NEG    1418
POS     937
Name: count, dtype: int64
microsoft: 3909 comments
sentiment
NEU    2743
NEG     788
POS     378
Name: count, dtype: int64
amazon: 2933 comments
sentiment
NEU    2000
NEG     649
POS     284
Name: count, dtype: int64
Saudi Arabian Oil Co DataFrame is None.
intel: 8063 comments
sentiment
NEU    5500
NEG    1861
POS     702
Name: count, dtype: int64
tesla: 15809 comments
sentiment
NEU    10258
NEG     4097
POS     1454
Name: count, dtype: int64
MARA Holdings: 240 comments
sentiment
NEU    170
POS     39
NEG     31
Name: count, dtype: int64
